# Dataset Challenges
> “Whoever fights monsters should see to it that in the process he does not become a monster. And if you gaze long enough into an abyss, the abyss will gaze back into you.”
>
> ― Friedrich Nietzsche

In [ ]:
# default_exp dataset_challenges

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


To model real-world use cases better, we need:
* redundant/duplicated data;
* noisy labels (emulating noisy oracles);
* class imbalance;
* out-of-distribution data/outliers included in the unlabelled data;
* noisy or ambiguous samples.

RepeatedMNIST takes care of the first and last challenge in a very specific way. 
This chapter takes care of the other ones.

In [ ]:
# exports

import bisect
from dataclasses import dataclass
from typing import List, Optional, Union, Dict, Type

import numpy as np
import torch
import torch.utils.data as data
import torchvision.datasets

from batchbald_redux.fast_mnist import FastMNIST, FastFashionMNIST

Monkey patch data.Dataset to have better doc strings:

## A new, more extension friendly Dataset base class

In [ ]:
# exports


def _wrap_alias(dataset: data.Dataset):
    if not isinstance(dataset, NamedDataset):
        return repr(dataset)
    return f"({dataset.alias})"


class AliasDataset(data.Dataset):
    """
    A dataset with an easier to understand alias. And convenience operators and methods.
    """

    dataset: data.Dataset
    alias: str

    def __init__(self, dataset: data.Dataset, alias: str):
        self.dataset = dataset
        self.alias = alias

    def __getitem__(self, idx):
        return self.dataset[idx]

    def __len__(self):
        return len(self.dataset)

    def __repr__(self):
        return self.alias

    def __add__(self, other):
        assert get_num_classes(self.dataset) == get_num_classes(other)
        return AliasDataset(data.ConcatDataset([self, other]), f"{_wrap_alias(self)} + {_wrap_alias(other)}")

    def __mul__(self, factor):
        if int(factor) == factor:
            return RepeatedDataset(self, num_repeats=factor)
        if factor < 1:
            return RandomSubsetDataset(self, factor=factor, seed=0)
        return RepeatedDataset(self, num_repeats=int(factor)) + RandomSubsetDataset(self, factor=factor % 1, seed=0)

    def __rmul__(self, factor):
        return self * factor

    def get_base_dataset_index(self, index):
        return get_base_dataset_index(self.dataset, index)

    def get_num_classes(self):
        return get_num_classes(self.dataset)

    def get_target(self, index, *, device=None):
        return get_target(self.dataset, index)

    def get_targets(self, device=None):
        return get_targets(self.dataset)

    def subset(self, indices: list):
        return SubsetAliasDataset(self, indices)

    def imbalance(self, *, class_counts: list, seed: int):
        return ImbalancedDataset(self, class_counts=class_counts, seed=seed)

    def override_targets(
            self, *, targets: Union[list, torch.Tensor], indices: list = None, num_classes: int = None
    ):
        if not indices:
            return ReplaceTargetsDataset(self, targets=targets, num_classes=num_classes)
        return OverrideTargetsDataset(self, indices=indices, targets=targets, num_classes=num_classes)

    def corrupt_labels(
            self, *, size_corrupted: Union[float, int], seed: int, num_classes: int = None, device=None
    ):
        return CorruptedLabelsDataset(
            self, size_corrupted=size_corrupted, seed=seed, num_classes=num_classes, device=device
        )

    def randomize_labels(self, *, seed: int, num_classes: int = None, device=None):
        return RandomLabelsDataset(self, seed=seed, num_classes=num_classes, device=device)

    def imbalance_dataset_split(
            self, *, num_classes: int, majority_percentage: int, minority_percentage: int, seed: int
    ):
        return ImbalancedClassSplitDataset(
            self,
            num_classes=num_classes,
            majority_percentage=majority_percentage,
            minority_percentage=minority_percentage,
            seed=seed,
        )

    def one_hot(self, *, dtype=None, device=None):
        return OneHotDataset(self, dtype=dtype, device=device)

    def constant_target(self, target: torch.Tensor, *, num_classes=None):
        return ConstantTargetDataset(dataset=self, target=target, num_classes=num_classes)

    def uniform_target(self, *, num_classes=None, dtype=None, device=None):
        return UniformTargetDataset(self, num_classes=num_classes, dtype=dtype, device=device)

## Auxiliary Methods

We sometimes want to:

* obtain the base dataset for a certain index (for OOD detection)
* get only a target for a certain index; or
* get all targets for a dataset.

In [ ]:
# exports


@dataclass
class DatasetIndex:
    dataset: data.Dataset
    index: int


def get_base_dataset_index(dataset: data.Dataset, index: int) -> DatasetIndex:
    if isinstance(dataset, data.ConcatDataset):
        if index < 0:
            if -index > len(dataset):
                raise ValueError("absolute value of index should not exceed dataset length")
            index = len(dataset) + index
        dataset_idx = bisect.bisect_right(dataset.cumulative_sizes, index)
        if dataset_idx == 0:
            sample_idx = index
        else:
            sample_idx = index - dataset.cumulative_sizes[dataset_idx - 1]
        return get_base_dataset_index(dataset.datasets[dataset_idx], int(sample_idx))
    elif isinstance(dataset, data.Subset):
        return get_base_dataset_index(dataset.dataset, int(dataset.indices[index]))
    elif isinstance(dataset, data.TensorDataset):
        return DatasetIndex(dataset, int(index))
    elif isinstance(dataset, torchvision.datasets.MNIST):
        return DatasetIndex(dataset, int(index))
    elif isinstance(dataset, torchvision.datasets.CIFAR10):
        return DatasetIndex(dataset, int(index))
    elif isinstance(dataset, torchvision.datasets.SVHN):
        return DatasetIndex(dataset, int(index))
    elif isinstance(dataset, AliasDataset):
        return dataset.get_base_dataset_index(index)
    else:
        raise NotImplementedError(f"Unrecognized dataset {dataset}!")


def get_num_classes(dataset: data.Dataset) -> int:
    if isinstance(dataset, data.ConcatDataset):
        return get_num_classes(dataset.datasets[0])
    elif isinstance(dataset, data.Subset):
        return get_num_classes(dataset.dataset)
    elif isinstance(dataset, torchvision.datasets.MNIST):
        return len(dataset.classes)
    elif isinstance(dataset, torchvision.datasets.CIFAR10):
        return len(dataset.classes)
    elif isinstance(dataset, torchvision.datasets.SVHN):
        return 10
    elif isinstance(dataset, AliasDataset):
        return dataset.get_num_classes()
    elif hasattr(dataset, "num_classes"):
        return dataset.num_classes
    else:
        raise NotImplementedError(f"Unrecognized dataset {dataset}!")


def get_target(dataset, index: int, *, device=None):
    if isinstance(dataset, data.ConcatDataset):
        if index < 0:
            if -index > len(dataset):
                raise ValueError("absolute value of index should not exceed dataset length")
            index = len(dataset) + index
        dataset_idx = bisect.bisect_right(dataset.cumulative_sizes, index)
        if dataset_idx == 0:
            sample_idx = index
        else:
            sample_idx = index - dataset.cumulative_sizes[dataset_idx - 1]
        target = get_target(dataset.datasets[dataset_idx], sample_idx, device=device)
    elif isinstance(dataset, data.Subset):
        target = get_target(dataset.dataset, dataset.indices[index], device=device)
    elif isinstance(dataset, data.TensorDataset):
        target = dataset.tensors[1][index]
    elif isinstance(dataset, torchvision.datasets.MNIST):
        target = dataset.targets[index]
    elif isinstance(dataset, torchvision.datasets.CIFAR10):
        target = dataset.targets[index]
    elif isinstance(dataset, torchvision.datasets.SVHN):
        target = dataset.labels[index]
    elif isinstance(dataset, AliasDataset):
        target = dataset.get_target(index, device=device)
    else:
        raise NotImplementedError(f"Unrecognized dataset {dataset}!")
    return torch.as_tensor(target, device=device)


def get_targets(dataset, *, device=None) -> torch.Tensor:
    if isinstance(dataset, data.ConcatDataset):
        # Move all tensors to the same device for torch.cat. When in doubt, use the cpu.
        return torch.cat([get_targets(sub_dataset, device=device or "cpu") for sub_dataset in dataset.datasets])
    elif isinstance(dataset, data.Subset):
        return get_targets(dataset.dataset, device=device)[torch.as_tensor(dataset.indices)]
    elif isinstance(dataset, data.TensorDataset):
        return torch.as_tensor(dataset.tensors[1], device=device)
    elif isinstance(dataset, torchvision.datasets.MNIST):
        return torch.as_tensor(dataset.targets, device=device)
    elif isinstance(dataset, torchvision.datasets.CIFAR10):
        return torch.as_tensor(dataset.targets, device=device)
    elif isinstance(dataset, torchvision.datasets.SVHN):
        return torch.as_tensor(dataset.labels, device=device)
    elif isinstance(dataset, AliasDataset):
        return dataset.get_targets(device=device)

    raise NotImplementedError(f"Unrecognized dataset {dataset} with type {type(dataset)}!")

In [ ]:
# exports


class SubsetAliasDataset(AliasDataset):
    indices: list

    def __init__(self, dataset: data.Dataset, indices: list, alias: str = None):
        alias = alias or f"{dataset}[{indices}]"

        super().__init__(dataset, alias)
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    def get_base_dataset_index(self, index) -> DatasetIndex:
        return get_base_dataset_index(self.dataset, self.indices[index])

    def get_num_classes(self) -> int:
        return get_num_classes(self.dataset)

    def get_target(self, index, *, device=None) -> torch.Tensor:
        return get_target(self.dataset, self.indices[index], device=device)

    def get_targets(self, device=None) -> torch.Tensor:
        return get_targets(self.dataset, device=device)[torch.as_tensor(self.indices)]


class ReplaceTargetsDataset(AliasDataset):
    targets: torch.Tensor
    num_classes: int

    def __init__(self, dataset: data.Dataset, targets: torch.Tensor, *, num_classes: int = None, alias: str = None):
        assert len(dataset) == len(targets)

        num_classes = num_classes or get_num_classes(dataset)
        alias = alias or f"{dataset} | replace_targets{dict(targets=targets, num_classes=num_classes)}"

        super().__init__(dataset, alias)
        self.num_classes = num_classes
        self.targets = targets

    def __getitem__(self, index):
        data, _ = self.dataset[index]
        return data, self.targets[index]

    def get_num_classes(self):
        return self.num_classes

    def get_target(self, index, *, device=None):
        return torch.as_tensor(self.targets[index], device=device)

    def get_targets(self, device=None):
        return torch.as_tensor(self.targets, device=device)


class NamedDataset(AliasDataset):
    def __init__(self, dataset: data.Dataset, name: str):
        super().__init__(dataset, repr(name))

    def get_base_dataset_index(self, index) -> DatasetIndex:
        # NamedDatasets are a leaf for all purposes.
        return DatasetIndex(self, index)

In [ ]:
from batchbald_redux.fast_mnist import FastMNIST

MNIST = NamedDataset(FastMNIST(root="data/", download=True, device="cpu"), name="MNISTDataset")

MNIST

'MNISTDataset'

## Noisy Labels

In [ ]:
# exports


class OverrideTargetsDataset(AliasDataset):
    reverse_indices: dict
    new_targets: list
    num_classes: int

    def __init__(
        self, dataset: data.Dataset, *, indices: list, targets: Union[list, torch.Tensor], num_classes: int = None
    ):
        assert len(indices) == len(targets)

        self.reverse_indices = {idx: rank for rank, idx in enumerate(indices)}
        self.targets = targets
        self.num_classes = num_classes or get_num_classes(self.dataset)

        super().__init__(
            dataset,
            f"{dataset} | override_targets{dict(indices=indices, targets=targets, num_classes=num_classes)}",
        )

    def __getitem__(self, index):
        data, target = self.dataset[index]

        if index not in self.reverse_indices:
            return data, target

        reverse_index = self.reverse_indices[index]
        new_target = self.targets[reverse_index]
        return data, new_target

    def get_num_classes(self):
        return self.num_classes

    def get_target(self, index, *, device=None):
        if index not in self.reverse_indices:
            return get_target(self.dataset, index, device=device)

        ridx = self.reverse_indices[index]
        new_target = self.targets[ridx]
        return torch.as_tensor(new_target, device=device)

    def get_targets(self, device=None):
        targets = torch.clone(get_targets(self.dataset))
        targets[list(self.reverse_indices.keys())] = torch.as_tensor(self.targets)
        return torch.as_tensor(targets, device=device)


class CorruptedLabelsDataset(AliasDataset):
    options: dict
    implementation: OverrideTargetsDataset

    def __init__(
        self,
        dataset: data.Dataset,
        *,
        size_corrupted: Union[float, int],
        seed: int,
        num_classes: int = None,
        device=None,
    ):
        num_classes = num_classes or get_num_classes(dataset)
        options = dict(size_corrupted=size_corrupted, num_classes=num_classes, seed=seed)

        super().__init__(dataset, f"{dataset} | corrupt_labels{options}")
        self.options = options
        self.num_classes = num_classes

        N = len(dataset)

        if size_corrupted > 1:
            num_corrupted = size_corrupted
        else:
            num_corrupted = int(N * size_corrupted)

        generator = np.random.default_rng(seed)
        indices = generator.choice(N, size=num_corrupted, replace=False)
        new_targets = generator.choice(num_classes, size=num_corrupted, replace=True)

        self.implementation = OverrideTargetsDataset(
            dataset, indices=indices, targets=torch.as_tensor(new_targets, device=device), num_classes=num_classes
        )

    def __getitem__(self, idx):
        return self.implementation[idx]

    def __len__(self):
        return len(self.implementation)

    def get_num_classes(self):
        return self.num_classes

    def get_target(self, index, *, device=None):
        return self.implementation.get_target(index, device=device)

    def get_targets(self, device=None):
        return self.implementation.get_targets(device=device)


class RandomLabelsDataset(ReplaceTargetsDataset):
    options: dict

    def __init__(self, dataset: data.Dataset, *, seed: int, num_classes: int = None, device=None):
        num_classes = num_classes or get_num_classes(dataset)
        options = dict(num_classes=num_classes, seed=seed)

        generator = np.random.default_rng(seed)
        N = len(dataset)
        targets = torch.as_tensor(generator.choice(num_classes, size=N, replace=True), device=device)

        super().__init__(dataset, targets, num_classes=num_classes, alias=f"{dataset} | randomize_labels{options}")
        self.options = options

### Example

In [ ]:
zero_dataset = NamedDataset(data.TensorDataset(torch.zeros(10), torch.zeros(10)), "ZeroDataset")

corrupted_labels_dataset = CorruptedLabelsDataset(zero_dataset, size_corrupted=0.5, num_classes=10, seed=1)

assert list(corrupted_labels_dataset) == [
    (torch.tensor(0.0), torch.tensor(8)),
    (torch.tensor(0.0), torch.tensor(0.0)),
    (torch.tensor(0.0), torch.tensor(8)),
    (torch.tensor(0.0), torch.tensor(3)),
    (torch.tensor(0.0), torch.tensor(0.0)),
    (torch.tensor(0.0), torch.tensor(0.0)),
    (torch.tensor(0.0), torch.tensor(4)),
    (torch.tensor(0.0), torch.tensor(0.0)),
    (torch.tensor(0.0), torch.tensor(2)),
    (torch.tensor(0.0), torch.tensor(0.0)),
]

corrupted_labels_dataset

'ZeroDataset' | corrupt_labels{'size_corrupted': 0.5, 'num_classes': 10, 'seed': 1}

In [ ]:
corrupted_dataset = RandomLabelsDataset(zero_dataset, num_classes=10, seed=2)

assert list(corrupted_dataset) == [
    (torch.tensor(0.0), torch.tensor(8)),
    (torch.tensor(0.0), torch.tensor(2)),
    (torch.tensor(0.0), torch.tensor(1)),
    (torch.tensor(0.0), torch.tensor(2)),
    (torch.tensor(0.0), torch.tensor(4)),
    (torch.tensor(0.0), torch.tensor(8)),
    (torch.tensor(0.0), torch.tensor(4)),
    (torch.tensor(0.0), torch.tensor(0)),
    (torch.tensor(0.0), torch.tensor(3)),
    (torch.tensor(0.0), torch.tensor(6)),
]

corrupted_dataset

'ZeroDataset' | randomize_labels{'num_classes': 10, 'seed': 2}

## Class Imbalances

In [ ]:
# exports


def get_class_indices_by_class(
    dataset: data.Dataset, *, class_counts: list, generator: np.random.Generator
) -> Dict[int, int]:
    class_counts = list(class_counts)

    subset_indices = {label: [] for label in range(len(class_counts))}

    remaining_samples = sum(class_counts)

    indices = generator.permutation(len(dataset))
    targets = get_targets(dataset)
    for index in indices:
        target = targets[index].item()

        if class_counts[target] > 0:
            subset_indices[target].append(index)
            class_counts[target] -= 1
            remaining_samples -= 1

            if remaining_samples <= 0:
                break

    return subset_indices


def get_class_indices(dataset: data.Dataset, *, class_counts: list, generator: np.random.Generator) -> List[int]:
    indices_by_class = get_class_indices_by_class(dataset=dataset, class_counts=class_counts, generator=generator)
    return [index for by_class in indices_by_class.values() for index in by_class]


def get_balanced_sample_indices(dataset: data.Dataset, *, num_classes, samples_per_class, seed: int) -> List[int]:
    class_counts = [samples_per_class] * num_classes
    generator = np.random.default_rng(seed)

    return get_class_indices(dataset, class_counts=class_counts, generator=generator)


def get_balanced_sample_indices_by_class(
    dataset: data.Dataset, *, num_classes, samples_per_class, seed: int
) -> Dict[int, int]:
    class_counts = [samples_per_class] * num_classes
    generator = np.random.default_rng(seed)

    return get_class_indices_by_class(dataset, class_counts=class_counts, generator=generator)

In [ ]:
# exports


class ImbalancedDataset(SubsetAliasDataset):
    options: dict

    def __init__(self, dataset: data.Dataset, *, class_counts: list, seed: int):
        options = dict(class_counts=class_counts, seed=seed)
        generator = np.random.default_rng(seed)
        indices = get_class_indices(dataset, class_counts=class_counts, generator=generator)

        super().__init__(dataset, indices, f"ImbalancedDataset(dataset={dataset}, {options})")
        self.options = options


class ImbalancedClassSplitDataset(SubsetAliasDataset):
    options: dict

    def __init__(
        self, dataset: data.Dataset, *, num_classes: int, majority_percentage: int, minority_percentage: int, seed: int
    ):
        assert (num_classes % 2) == 0

        num_samples_per_class = len(dataset) // num_classes
        num_samples_majority = num_samples_per_class * majority_percentage // 100
        num_samples_minority = num_samples_per_class * minority_percentage // 100

        generator = np.random.default_rng(seed)

        class_counts = [num_samples_majority] * (num_classes // 2) + [num_samples_minority] * (num_classes // 2)
        class_counts = generator.permuted(class_counts)

        indices = get_class_indices(dataset, class_counts=class_counts, generator=generator)

        options = dict(
            num_classes=num_classes,
            majority_percentage=majority_percentage,
            minority_percentage=minority_percentage,
            seed=seed,
            class_counts=class_counts,
        )
        super().__init__(dataset, indices, f"ImbalancedDataset(dataset={dataset}, {options})")
        self.options = options

### Example

In [ ]:
three_dataset = NamedDataset(data.TensorDataset(torch.arange(9), torch.as_tensor(list(range(3)) * 3)), "123")

imbalanced_indices = get_class_indices(three_dataset, class_counts=[3, 0, 0], generator=np.random.default_rng())

three_dataset[imbalanced_indices][0]

tensor([6, 3, 0])

In [ ]:
imbalanced_dataset = ImbalancedDataset(three_dataset, class_counts=[1, 2, 3], seed=2)
print(imbalanced_dataset[:][1])
imbalanced_dataset

tensor([0, 1, 1, 2, 2, 2])


ImbalancedDataset(dataset='123', {'class_counts': [1, 2, 3], 'seed': 2})

In [ ]:
ImbalancedClassSplitDataset(MNIST, num_classes=10, majority_percentage=80, minority_percentage=20, seed=1)

ImbalancedDataset(dataset='MNISTDataset', {'num_classes': 10, 'majority_percentage': 80, 'minority_percentage': 20, 'seed': 1, 'class_counts': array([1200, 4800, 1200, 4800, 4800, 4800, 1200, 1200, 1200, 4800])})

## Mixing in OOD data

In [ ]:
# exports

# Convert label dataset to one hot
class OneHotDataset(ReplaceTargetsDataset):
    options: dict

    def __init__(self, dataset: data.Dataset, *, num_classes=None, dtype=None, device=None):
        num_classes = num_classes or get_num_classes(dataset)
        options = dict(num_classes=num_classes)

        N = len(dataset)
        targets = torch.zeros(N, num_classes, dtype=dtype, device=device)
        for i, label in enumerate(get_targets(dataset)):
            targets[i, label.item()] = 1.0

        super().__init__(dataset, targets, num_classes=num_classes, alias=f"{dataset} | one_hot_targets{options}")
        self.options = options


class RepeatedDataset(AliasDataset):
    def __init__(self, dataset: data.Dataset, *, num_repeats: int):
        self.num_repeats = num_repeats

        super().__init__(dataset, f"{_wrap_alias(dataset)}x{num_repeats}")

    def __getitem__(self, idx):
        if idx > len(self):
            return self.dataset[idx]

        return self.dataset[idx % len(self.dataset)]

    def __len__(self):
        return len(self.dataset) * self.num_repeats

    def get_base_dataset_index(self, index):
        return get_base_dataset_index(self.dataset, index % len(self.dataset))

    def get_target(self, index, *, device=None):
        return get_target(self.dataset, index % len(self.dataset), device=device)

    def get_targets(self, device=None):
        return get_targets(self.dataset, device=device).repeat(self.num_repeats)


class RandomSubsetDataset(SubsetAliasDataset):
    options: dict

    def __init__(self, dataset: data.Dataset, *, size: Optional[int] = None, factor: Optional[float] = None, seed: int):
        assert ((size is not None) or (factor is not None)) and not (size is None and factor is None)
        if size is not None:
            subset_size = size
            if seed == 0:
                alias = f"{_wrap_alias(dataset)}[:{size}]"
            else:
                alias = f"{_wrap_alias(dataset)}[:{size};seed={seed}]"
        elif factor is not None:
            subset_size = int(len(dataset) * factor)
            if seed == 0:
                alias = f"{_wrap_alias(dataset)}~x{factor}"
            else:
                alias = f"{_wrap_alias(dataset)}~x{factor} (seed={seed})"

        N = len(dataset)
        # noinspection PyUnboundLocalVariable
        assert subset_size < N

        generator = np.random.default_rng(seed)
        indices = generator.choice(N, size=subset_size, replace=False)

        # noinspection PyUnboundLocalVariable
        super().__init__(dataset, indices, alias)
        self.options = dict(size=size, factor=factor, seed=seed)


class ConstantTargetDataset(AliasDataset):
    target: torch.Tensor

    def __init__(self, dataset: data.Dataset, target: torch.Tensor, num_classes=None):
        num_classes = num_classes or get_num_classes(dataset)
        super().__init__(dataset, f"{dataset} | constant_target{dict(target=target, num_classes=num_classes)}")
        self.target = target
        self.num_classes = num_classes

    def __getitem__(self, idx):
        data, _ = self.dataset[idx]
        return data, self.target

    def get_num_classes(self):
        return self.num_classes

    def get_target(self, index, *, device=None):
        return torch.as_tensor(self.target, device=device)

    def get_targets(self, device=None):
        return torch.as_tensor(self.target, device=device).expand(len(self), *self.target.shape)


def UniformTargetDataset(dataset: data.Dataset, *, num_classes: int = None, dtype=None, device=None):
    num_classes = num_classes or get_num_classes(dataset)
    target = torch.ones(num_classes, dtype=dtype, device=device) / num_classes
    result = ConstantTargetDataset(dataset, target, num_classes=num_classes)
    result.options = dict(num_classes=num_classes)
    result.alias = f"{dataset} | uniform_targets{result.options}"
    return result

To an OOD dataset, one can either use:
```
MNIST+OOD*0.5
```
and then use `get_base_dataset(dataset, index) == OOD` to check whether a picked sample is OOD (see below).

Alternatively, we can use:
```
OneHotDataset(MNIST) + UniformTargetDataset(OOD * 0.5)
```

### Example

In [ ]:
MNIST * 0.1

('MNISTDataset')~x0.1

In [ ]:
MNIST * 3

('MNISTDataset')x3

In [ ]:
MNIST * 3.5

('MNISTDataset')x3 + ('MNISTDataset')~x0.5

In [ ]:
one_hot_MNIST = OneHotDataset(MNIST * 0.1)
print(one_hot_MNIST[0][1])

one_hot_MNIST

tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])


('MNISTDataset')~x0.1 | one_hot_targets{'num_classes': 10}

In [ ]:
UniformTargetDataset(MNIST, num_classes=10)

'MNISTDataset' | uniform_targets{'num_classes': 10}

## Noisy Samples

The problem is that for large datasets, precomputing the noise to be added can use up a lot of memory (doubling the dataset size). Creating a new random generator for each sample is too slow, so it might be worth creating an entirely new dataset.

However, we support exporting and importing datasets, so simply storing and loading the dataset is an option.

> Tip: Do not use this on very large datasets (e.g. ImageNet)... :)

In [ ]:
# exports


class AdditiveGaussianNoise(AliasDataset):
    noise: torch.Tensor
    options: dict

    def __init__(self, dataset: data.Dataset, sigma: float, *, max_num_noise_samples=None):
        sample = dataset[0][0]

        # Goal: easily add redundancy without OOM.
        if max_num_noise_samples is None:
            # A "big" prime, so it is unlikely we will repeat the noise
            max_num_noise_samples = 16273

        num_noise_samples = min(len(dataset), max_num_noise_samples)

        self.noise = sigma * torch.randn(num_noise_samples, *sample.shape, device=sample.device)
        self.options = dict(sigma=sigma, max_num_noise_samples=max_num_noise_samples)

        super().__init__(dataset, f"{dataset} + 𝓝(0;σ={sigma})")

    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        return sample + self.noise[idx % len(self.noise)], target

In [ ]:
noisy_zero = AdditiveGaussianNoise(zero_dataset, sigma=0.1, max_num_noise_samples=3)

print(noisy_zero)

list(noisy_zero)

'ZeroDataset' + 𝓝(0;σ=0.1)


[(tensor(0.0766), tensor(0.)),
 (tensor(0.2374), tensor(0.)),
 (tensor(0.0628), tensor(0.)),
 (tensor(0.0766), tensor(0.)),
 (tensor(0.2374), tensor(0.)),
 (tensor(0.0628), tensor(0.)),
 (tensor(0.0766), tensor(0.)),
 (tensor(0.2374), tensor(0.)),
 (tensor(0.0628), tensor(0.)),
 (tensor(0.0766), tensor(0.))]

## Exporting Datasets

Finally, to make it easier to use datasets across Python versions, we allow dataset exports.

In [ ]:
# exports


def dataset_to_tensors(dataset):
    samples = []
    targets = []

    for sample, target in dataset:
        samples.append(sample.to(device="cpu", non_blocking=True))
        targets.append(target.to(device="cpu", non_blocking=True))

    samples = torch.stack(samples)
    targets = torch.stack(targets)

    return samples, targets


def get_dataset_state_dict(dataset):
    dataset_alias = repr(dataset)

    samples, targets = dataset_to_tensors(dataset)

    num_classes = get_num_classes(dataset)

    state_dict = dict(alias=dataset_alias, samples=samples, targets=targets, num_classes=num_classes)

    return state_dict


class ImportedDataset(AliasDataset):
    def __init__(self, state_dict):
        tensor_dataset = data.TensorDataset(state_dict["samples"], state_dict["targets"])
        super().__init__(tensor_dataset, state_dict["alias"])
        self.num_classes = state_dict["num_classes"]

    def get_num_classes(self):
        return self.num_classes


def save_dataset(dataset: data.Dataset, f, **kwargs):
    torch.save(get_dataset_state_dict(dataset), f, **kwargs)


def load_dataset(f, map_location=None, **kwargs):
    state_dict = torch.load(f, map_location=map_location, **kwargs)
    dataset = ImportedDataset(state_dict)
    return dataset

In [ ]:
tensor_dataset = data.TensorDataset(torch.arange(0, 10), torch.arange(90, 100))
tensor_dataset.num_classes = 100
linear_dataset = NamedDataset(tensor_dataset, "LinearDataset")

samples, targets = dataset_to_tensors(linear_dataset)

assert all(samples == torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
assert all(targets == torch.tensor([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]))

In [ ]:
get_dataset_state_dict(linear_dataset)

{'alias': "'LinearDataset'",
 'samples': tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'targets': tensor([90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 'num_classes': 100}

In [ ]:
save_dataset(linear_dataset, "linear.dataset")

loaded_linear_dataset = load_dataset("linear.dataset")

loaded_samples, loaded_targets = dataset_to_tensors(loaded_linear_dataset)

assert all(loaded_samples == samples)
assert all(loaded_targets == targets)
assert get_num_classes(loaded_linear_dataset) == 100

## Repeated MNIST

In [ ]:
# exports


def create_named_mnist(mnist_type: Union[Type[FastMNIST], Type[FastFashionMNIST]], root: str,
           *,
        train: bool = True,
        download: bool = False,
        device = None
        ):
    dataset = mnist_type(root=root, train=train, download=download, device=device)
    name = f"{mnist_type.__name__} {'Train' if train else 'Test'} ({len(dataset)} samples)"
    return NamedDataset(dataset, name)


def create_repeated_MNIST_dataset(*, device=None, num_repetitions: int = 3, add_noise: bool = True):
    # num_classes = 10, input_size = 28

    train_dataset = NamedDataset(FastMNIST("data", train=True, download=True, device=device), "FastMNIST (Train)")

    rmnist_train_dataset = train_dataset
    if num_repetitions > 1:
        rmnist_train_dataset = train_dataset * num_repetitions

    if add_noise:
        rmnist_train_dataset = AdditiveGaussianNoise(rmnist_train_dataset, 0.1)

    test_dataset = NamedDataset(FastMNIST("data", train=False, device=device), "FastMNIST (Test)")

    return rmnist_train_dataset, test_dataset


def create_MNIST_dataset(device=None):
    return create_repeated_MNIST_dataset(num_repetitions=1, add_noise=False, device=device)

In [ ]:
rmnist_example = create_repeated_MNIST_dataset(device="cpu", num_repetitions=2, add_noise=True)
rmnist_example

('FastMNIST (Train)'x2 + 𝓝(0;σ=0.1), 'FastMNIST (Test)')

In [ ]:
len(get_targets(rmnist_example[0]))

120000

In [ ]:
rmnist_example[0] + rmnist_example[1]

('FastMNIST (Train)'x2 + 𝓝(0;σ=0.1)) + 'FastMNIST (Test)'

In [ ]:
MNIST.one_hot() * 2

('MNISTDataset' | one_hot_targets{'num_classes': 10})x2